In [17]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain


from langchain.chains.router import MultiPromptChain
from  langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE



In [18]:
fis_template = ChatPromptTemplate.from_template(
    """ 
    Você é um Professor de fisica muito inteligente.
    Você é otimo em responder perguntas sobre fisica concisa e facil de entender.
    Quando você não sabe a resposta, você admiti que não sabe.
    
    Aqui está a pergunta: {input}
    
    """
)

mat_template = ChatPromptTemplate.from_template(
    """
    Você é um Professor de matematica muito inteligente.
    Você é otimo em responder perguntas sobre matematica concisa e facil de entender.
    Quando você não sabe a resposta, você admiti que não sabe.

    Aqui está a pergunta: {input}

    """
)

hist_template = ChatPromptTemplate.from_template(
    """
    Você é um Professor de historia muito inteligente.
    Você é otimo em responder perguntas sobre historia concisa e facil de entender.
    Quando você não sabe a resposta, você admiti que não sabe.
    
    Aqui está a pergunta: {input}
    """  
    
    )



In [19]:
prompt_infos = [
    {
        "name": "Fisica",
        "description": "Ideal para responder perguntas sobre fisica",
        "prompt": fis_template,
    },
    {
        "name": "Matematica",
        "description": "Ideal para responder perguntas sobre matematica",
        "prompt": mat_template,
    },
    {
        "name": "Historia",
        "description": "Ideal para responder perguntas sobre historia",
        "prompt": hist_template,
    },
]

In [20]:
chat = ChatOpenAI()

In [21]:
chains_destino = {}

for info in prompt_infos:
    chain = LLMChain(llm=chat,  prompt=info["prompt"], verbose=True)
    chains_destino[info["name"]] = chain    

In [ ]:
destinos =[f"{p["name"]}: {p["description"]} " for p in prompt_infos]
destinos_str = "\n".join(destinos)
print(destinos_str)

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinos_str
)
print(router_template)

In [24]:
router_template = PromptTemplate(
    template= router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(chat, router_template, verbose=True)

In [ ]:
default_prompt = ChatPromptTemplate.from_template(
    "{input}"
)

default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destino,
    default_chain=default_chain,
    verbose=True
)

In [ ]:
chain.invoke({"input": "Quem descobriu o Brasil?"})

In [ ]:
"""
Codigo para router de chains
passando os parametros de entrada e saida
"""